This is the notebook used to understand how the block encoding can be generalized

In [1]:
from mps_ND import NDMPS
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt 
from utils_ND import *

/Users/maxge/miniconda3/envs/QEL/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
def get_factorlist(shape):
    # todo rename to actually get_block_sizes
    factor_lists = []
    for dim_size in shape:
        if dim_size == 1:
            # Handle dimension of size 1 by assigning a factor of 1
            factors = [1]
        else:
            factors_dict = factorint(dim_size)
            # Extract prime factors and repeat them according to their exponents
            factors = []
            for prime, exponent in sorted(factors_dict.items()):
                factors.extend([prime] * exponent)
        # Sort the factors in ascending order
        factors_sorted = sorted(factors)
        factor_lists.append(factors_sorted)
    
    # Step 2: Balancing the number of factors across all dimensions
    # Determine the minimum number of factors among all dimensions
    min_factors = min(len(factors) for factors in factor_lists)
    
    # Balance factors by grouping the smallest factors in dimensions with more factors
    for idx, factors in enumerate(factor_lists):
        if len(factors) > min_factors:
            factor_lists[idx] = balance_factors(factors, min_factors)
        elif len(factors) < min_factors:
            # If a dimension has fewer factors, pad with 1s to reach min_factors
            # This effectively treats missing factors as trivial
            factor_lists[idx].extend([1] * (min_factors - len(factors)))
            factor_lists[idx] = sorted(factor_lists[idx])
        # If equal, do nothing
    for idx, list in enumerate(factor_lists):
        if idx%2 == 1:
            factor_lists[idx] = factor_lists[idx][::-1] 

    return np.array(factor_lists).T

def hierarchical_block_indexing(index, shape, block_sizes):
    num_levels = len(block_sizes)
    dim = len(index)
    # important only insert numpy arrays
    hierarchical_indices = []
    for level in range(num_levels):
        if level == 0:
            hierarchical_indices.append(np.floor(index/np.prod(block_sizes[1:], axis=0)))
        elif level == num_levels-1:
            hierarchical_indices.append(np.floor(np.mod(index, block_sizes[-1])))
        else:
            hierarchical_indices.append(np.floor(np.mod(index, np.prod(block_sizes[level:], axis=0))/np.prod(block_sizes[level+1:], axis=0)))
    return hierarchical_indices


def flatten_index(index, block_shape):
    """
    Converts a multi-dimensional index into a flattened index using row-major order (C-style).
    
    Parameters:
    - index: tuple of indices (i1, i2, ..., id)
    - block_shape: tuple representing the shape of the block (B1, B2, ..., Bd)
    
    Returns:
    - Flattened index (int)
    """
    if len(index) != len(block_shape):
        raise ValueError("Index and block shape must have the same number of dimensions.")
    
    # Compute the flattened index using numpy's ravel_multi_index equivalent logic
    flattened_index = sum(index[k] * np.prod(block_shape[k+1:]) for k in range(len(index)))
    
    return flattened_index.astype(int)

def return_flattened_idices(hierarchical_index, block_sizes):
    ids = []
    for i in range(len(block_sizes)):
        ids.append(flatten_index(hierarchical_index[i], block_sizes[i]))
    return ids

def stdId_to_hierarchical_index(stdId, shape):
    block_sizes = get_factorlist(shape)
    hier_ind = hierarchical_block_indexing(stdId, shape, block_sizes)
    return return_flattened_idices(hier_ind, block_sizes)




In [3]:
shape = (64,32,20,54)
block_sizes = get_factorlist(shape)
index = (5, 6,7,10)
hierarchical_index = hierarchical_block_indexing(index, shape, block_sizes)
flatten_ind = return_flattened_idices(hierarchical_index, block_sizes)
print(flatten_ind)

[np.int64(1), np.int64(45), np.int64(37)]


In [4]:
print(block_sizes)
print(block_sizes[-1:0:-1])
print(np.cumprod(block_sizes[-1:0:-1], axis =0)[::-1])

[[4 4 2 6]
 [4 4 2 3]
 [4 2 5 3]]
[[4 2 5 3]
 [4 4 2 3]]
[[16  8 10  9]
 [ 4  2  5  3]]


In [5]:
hierarchical_index

[array([0., 0., 0., 1.]), array([1., 3., 1., 0.]), array([1., 0., 2., 1.])]

In [6]:
stdId_to_hierarchical_index(index, shape)

[np.int64(1), np.int64(45), np.int64(37)]

In [7]:
qubit_size, kron = get_block_encoding_map_general(shape)
kron[index]

(np.int64(1), np.int64(45), np.int64(37))

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import quimb.tensor as qtn
from utils_ND import *
from scipy.fftpack import dct, idct
import time

def time_function(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        print(f"Time to run {func.__name__}: {elapsed_time:.4f} seconds")
        return result
    return wrapper


class NDMPS_old:
    def __init__(self, mps=None, qubit_size=None, encoding_map=None, norm=True, mode="Std", min_value = 0, max_value = 1):
        self.qubit_size = qubit_size
        self.encoding_map = encoding_map
        self.mps = mps
        self.norm = norm #Normalize matrix data
        #Compression mode 
        # "Std" standard Block Encoding
        # "DCT" discrete cosine fourier transform before compression
        self.mode = mode 
        self.min_value = min_value
        self.max_value = max_value
    
    @classmethod
    # @time_function
    def from_matrix(cls, matrix, norm = False, mode = "Std"):
        qubit_size, encoding_map = get_block_encoding_map_general(matrix.shape)

        #check for flags
        if norm:
            matrix = matrix / (np.linalg.norm(matrix))
        if mode == "DCT":
            matrix = dct(matrix, norm = "ortho")

        #initialize tensor
        contracted_tensor = np.empty(shape = tuple(qubit_size))

        #encode matrix data
        # start_nested_loop = time.time()
        it = np.nditer(matrix, flags=['multi_index'])
        for _ in it:
            contracted_tensor[encoding_map[it.multi_index]] = matrix[it.multi_index]
        
        # nested_loop_time = time.time() - start_nested_loop
        # print(f"Time for nested loops: {nested_loop_time:.4f} seconds")
        #put in MPS
        # start_mps_creation = time.time()
        mps = qtn.MatrixProductState.from_dense(contracted_tensor, dims = tuple(qubit_size))
        # mps_creation_time = time.time() - start_mps_creation
        # print(f"Time to create MPS from dense tensor: {mps_creation_time:.4f} seconds")
        #return class
        return cls(mps, qubit_size, encoding_map, norm, mode, np.min(matrix), np.max(matrix))

    # @time_function
    def compression_ratio(self):
        initial_N = np.prod(self.qubit_size)
        compressed_N = self.number_elements_in_MPS()
        # TODO: also implement the compression rate in bits / bits
        return compressed_N / initial_N
        
    # @time_function
    def compress(self, cutoff):
        """
        Compresses a Matrix Product State (MPS) by cutting bonds with a relative cutoff value.
        Arguments:
            cutoff (float): The relative cutoff value to use for bond compression.
        Returns:
            None
        """
        size = len(self.mps.sites)
        for i in np.arange(1, size):
            t1 = self.mps[i-1] # Tensor 1
            t2 = self.mps[i] # Tensor 2
            # Compress bond according to percentage * bond dimension
            qtn.tensor_compress_bond(t1, t2, cutoff = cutoff, cutoff_mode = "rel") 
    def continuous_compress(self, cutoff, print_ratio = True):
        compress_list = np.array([0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1]) * cutoff
        for c in compress_list:
            self.compress(c)
            if print_ratio:
                print(f"Compression ratio at {c}: {self.compression_ratio()}")


    # @time_function
    def number_elements_in_MPS(self):
        """
        Returns the number of tensor elements in the quimb MPS.
        Parameters:
            mps: quimb MatrixProductState object
        Returns:
            int: The total number of tensor elements in the MPS."""
        return sum(t.size for t in self.mps)
    
    # @time_function
    def mps_to_matrix(self):
        """
        Converts the compressed Matrix Product State (MPS) representation back to an image matrix.
        Arguments:
            None
        Returns:
            Compressed matrix
        """

        #conract mps
        contracted_mps = self.mps ^ ...

        #order tensor legs back
        for i in np.arange(len(contracted_mps.inds)):
            contracted_mps.moveindex("k"+str(i), i, inplace=True)
        
        #return in correct format

        recovered_tensor = np.empty(self.encoding_map.shape)
        it = np.nditer(recovered_tensor, flags=['multi_index'])
        for _ in it:
            recovered_tensor[it.multi_index] = contracted_mps.data[self.encoding_map[it.multi_index]]
        
        if self.mode == "Std":
            return recovered_tensor
        elif self.mode == "DCT":
            return idct(recovered_tensor, norm = "ortho")
        """
        -----
        legacy code for images
        if self.mode == "Std":
            img = [
            [contracted_mps.data[self.encoding_map[i,j]] for j in range(self.encoding_map.shape[1])] for i in range(self.encoding_map.shape[0])
        ]
            return rescale_image(img)

        elif self.mode == "DCT":
            img = idct([
                [contracted_mps.data[self.encoding_map[i,j]] \
                for j in range(self.encoding_map.shape[1])] for i in range(self.encoding_map.shape[0])
            ], norm = "ortho")
            return rescale_image(img)"""



In [9]:
print(block_sizes)
print(block_sizes[-1:0:-1])
print(np.cumprod(block_sizes[-1:0:-1], axis =0)[::-1])

[[4 4 2 6]
 [4 4 2 3]
 [4 2 5 3]]
[[4 2 5 3]
 [4 4 2 3]]
[[16  8 10  9]
 [ 4  2  5  3]]


In [10]:
np.ones((len(block_sizes)+1, len(block_sizes[0])), dtype = int)

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [ ]:
 

def time_function(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        print(f"Time to run {func.__name__}: {elapsed_time:.8f} seconds")
        return result
    return wrapper

@time_function
def get_factorlist(shape):
    # todo rename to actually get_block_sizes
    factor_lists = []
    for dim_size in shape:
        if dim_size == 1:
            # Handle dimension of size 1 by assigning a factor of 1
            factors = [1]
        else:
            factors_dict = factorint(dim_size)
            # Extract prime factors and repeat them according to their exponents
            factors = []
            for prime, exponent in sorted(factors_dict.items()):
                factors.extend([prime] * exponent)
        # Sort the factors in ascending order
        factors_sorted = sorted(factors)
        factor_lists.append(factors_sorted)
    
    # Step 2: Balancing the number of factors across all dimensions
    # Determine the minimum number of factors among all dimensions
    min_factors = min(len(factors) for factors in factor_lists)
    
    # Balance factors by grouping the smallest factors in dimensions with more factors
    for idx, factors in enumerate(factor_lists):
        if len(factors) > min_factors:
            factor_lists[idx] = balance_factors(factors, min_factors)
        elif len(factors) < min_factors:
            # If a dimension has fewer factors, pad with 1s to reach min_factors
            # This effectively treats missing factors as trivial
            factor_lists[idx].extend([1] * (min_factors - len(factors)))
            factor_lists[idx] = sorted(factor_lists[idx])
        # If equal, do nothing
    for idx, list in enumerate(factor_lists):
        if idx%2 == 1:
            factor_lists[idx] = factor_lists[idx][::-1] 

    factor_lists = np.array(factor_lists).T
    prod_block_sizes = np.ones((len(factor_lists)+1, len(factor_lists[0])), dtype = int)
    prod_block_sizes[1:-1] = np.cumprod(factor_lists[-1:0:-1], axis =0)[::-1]
    prod_block_sizes[0] = prod_block_sizes[0] * 1e100

    return factor_lists, prod_block_sizes

@time_function
def hierarchical_block_indexing(index, prod_block_sizes):
    return np.floor(np.mod(index, prod_block_sizes[:-1])/prod_block_sizes[1:])

@time_function
def hierarchical_block_indexing_old(index, shape, block_sizes):
    num_levels = len(block_sizes)
    dim = len(index)
    # important only insert numpy arrays
    hierarchical_indices = np.zeros((num_levels, dim))
    for level in range(num_levels):
        if level == 0:
            hierarchical_indices[0] = (np.floor(index/np.prod(block_sizes[1:], axis=0)))
        elif level == num_levels-1:
            hierarchical_indices[level] = (np.floor(np.mod(index, block_sizes[-1])))
        else:
            hierarchical_indices[level]= (np.floor(np.mod(index, np.prod(block_sizes[level:], axis=0))/np.prod(block_sizes[level+1:], axis=0)))
    return hierarchical_indices

@time_function
def flatten_index(index, block_shape):
    """
    Converts a multi-dimensional index into a flattened index using row-major order (C-style).
    
    Parameters:
    - index: tuple of indices (i1, i2, ..., id)
    - block_shape: tuple representing the shape of the block (B1, B2, ..., Bd)
    
    Returns:
    - Flattened index (int)
    """
    if len(index) != len(block_shape):
        raise ValueError("Index and block shape must have the same number of dimensions.")
    
    # Compute the flattened index using numpy's ravel_multi_index equivalent logic
    flattened_index = sum(index[k] * np.prod(block_shape[k+1:]) for k in range(len(index)))
    
    return flattened_index.astype(int)

@time_function
def return_flattened_idices(hierarchical_index, block_sizes):
    ids = np.empty(len(block_sizes))
    for i in range(len(block_sizes)):
        ids[i] = flatten_index(hierarchical_index[i], block_sizes[i])
    return ids

@time_function
def stdId_to_hierarchical_index(stdId, shape, block_sizes):
    hier_ind = hierarchical_block_indexing(stdId, shape, block_sizes)
    return return_flattened_idices(hier_ind, block_sizes)

class NDMPS:
    def __init__(self, mps=None, qubit_size=None, norm=True, mode="Std", min_value = 0, max_value = 1):
        self.qubit_size = qubit_size
        self.mps = mps
        self.norm = norm #Normalize matrix data
        #Compression mode 
        # "Std" standard Block Encoding
        # "DCT" discrete cosine fourier transform before compression
        self.mode = mode 
        self.min_value = min_value
        self.max_value = max_value
    
    @classmethod
    # @time_function
    def from_matrix(cls, matrix, norm = False, mode = "Std"):
        shape = matrix.shape
        qubit_size = np.prod(get_factorlist(shape), axis =1)
        block_sizes = get_factorlist(shape)

        #check for flags
        if norm:
            matrix = matrix / (np.linalg.norm(matrix))
        if mode == "DCT":
            matrix = dct(matrix, norm = "ortho")

        #initialize tensor
        contracted_tensor = np.empty(shape = tuple(qubit_size))

        #encode matrix data
        # start_nested_loop = time.time()
        it = np.nditer(matrix, flags=['multi_index'])
        for _ in it:
            contracted_tensor[tuple(stdId_to_hierarchical_index(it.multi_index, shape, block_sizes))] = matrix[it.multi_index]
        
        # nested_loop_time = time.time() - start_nested_loop
        # print(f"Time for nested loops: {nested_loop_time:.4f} seconds")
        #put in MPS
        # start_mps_creation = time.time()
        mps = qtn.MatrixProductState.from_dense(contracted_tensor, dims = tuple(qubit_size))
        # mps_creation_time = time.time() - start_mps_creation
        # print(f"Time to create MPS from dense tensor: {mps_creation_time:.4f} seconds")
        #return class
        return cls(mps, qubit_size, norm, mode, np.min(matrix), np.max(matrix))

    # @time_function
    def compression_ratio(self):
        initial_N = np.prod(self.qubit_size)
        compressed_N = self.number_elements_in_MPS()
        # TODO: also implement the compression rate in bits / bits
        return compressed_N / initial_N
        
    # @time_function
    def compress(self, cutoff):
        """
        Compresses a Matrix Product State (MPS) by cutting bonds with a relative cutoff value.
        Arguments:
            cutoff (float): The relative cutoff value to use for bond compression.
        Returns:
            None
        """
        size = len(self.mps.sites)
        for i in np.arange(1, size):
            t1 = self.mps[i-1] # Tensor 1
            t2 = self.mps[i] # Tensor 2
            # Compress bond according to percentage * bond dimension
            qtn.tensor_compress_bond(t1, t2, cutoff = cutoff, cutoff_mode = "rel") 
    def continuous_compress(self, cutoff, print_ratio = True):
        compress_list = np.array([0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1]) * cutoff
        for c in compress_list:
            self.compress(c)
            if print_ratio:
                print(f"Compression ratio at {c}: {self.compression_ratio()}")


    # @time_function
    def number_elements_in_MPS(self):
        """
        Returns the number of tensor elements in the quimb MPS.
        Parameters:
            mps: quimb MatrixProductState object
        Returns:
            int: The total number of tensor elements in the MPS."""
        return sum(t.size for t in self.mps)
    
    # @time_function
    def mps_to_matrix(self):
        """
        Converts the compressed Matrix Product State (MPS) representation back to an image matrix.
        Arguments:
            None
        Returns:
            Compressed matrix
        """

        #conract mps
        contracted_mps = self.mps ^ ...

        #order tensor legs back
        for i in np.arange(len(contracted_mps.inds)):
            contracted_mps.moveindex("k"+str(i), i, inplace=True)
        
        #return in correct format

        recovered_tensor = np.empty(self.encoding_map.shape)
        it = np.nditer(recovered_tensor, flags=['multi_index'])
        for _ in it:
            recovered_tensor[it.multi_index] = contracted_mps.data[self.encoding_map[it.multi_index]]
        
        if self.mode == "Std":
            return recovered_tensor
        elif self.mode == "DCT":
            return idct(recovered_tensor, norm = "ortho")
        """
        -----
        legacy code for images
        if self.mode == "Std":
            img = [
            [contracted_mps.data[self.encoding_map[i,j]] for j in range(self.encoding_map.shape[1])] for i in range(self.encoding_map.shape[0])
        ]
            return rescale_image(img)

        elif self.mode == "DCT":
            img = idct([
                [contracted_mps.data[self.encoding_map[i,j]] \
                for j in range(self.encoding_map.shape[1])] for i in range(self.encoding_map.shape[0])
            ], norm = "ortho")
            return rescale_image(img)"""

In [12]:
shape = (240,240,220)
block_sizes, prod_blocks = get_factorlist(shape)
index = (100, 100,100)
hierarchical_index = hierarchical_block_indexing_old(index, shape, block_sizes)
flatten_ind = return_flattened_idices(hierarchical_index, block_sizes)
print(hierarchical_index)
print(flatten_ind)

Time to run get_factorlist: 0.00027704 seconds
Time to run hierarchical_block_indexing_old: 0.00007033 seconds
Time to run flatten_index: 0.00003099 seconds
Time to run flatten_index: 0.00001597 seconds
Time to run flatten_index: 0.00001311 seconds
Time to run flatten_index: 0.00001287 seconds
Time to run return_flattened_idices: 0.00010276 seconds
[[1. 2. 0.]
 [1. 0. 1.]
 [0. 1. 4.]
 [0. 1. 1.]]
[14.  9.  9. 12.]


/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/85896759.py:56: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


In [64]:
def get_factorlist(shape):
    # todo rename to actually get_block_sizes
    factor_lists = []
    for dim_size in shape:
        if dim_size == 1:
            # Handle dimension of size 1 by assigning a factor of 1
            factors = [1]
        else:
            factors_dict = factorint(dim_size)
            # Extract prime factors and repeat them according to their exponents
            factors = []
            for prime, exponent in sorted(factors_dict.items()):
                factors.extend([prime] * exponent)
        # Sort the factors in ascending order
        factors_sorted = sorted(factors)
        factor_lists.append(factors_sorted)
    
    # Step 2: Balancing the number of factors across all dimensions
    # Determine the minimum number of factors among all dimensions
    min_factors = min(len(factors) for factors in factor_lists)
    
    # Balance factors by grouping the smallest factors in dimensions with more factors
    for idx, factors in enumerate(factor_lists):
        if len(factors) > min_factors:
            factor_lists[idx] = balance_factors(factors, min_factors)
        elif len(factors) < min_factors:
            # If a dimension has fewer factors, pad with 1s to reach min_factors
            # This effectively treats missing factors as trivial
            factor_lists[idx].extend([1] * (min_factors - len(factors)))
            factor_lists[idx] = sorted(factor_lists[idx])
        # If equal, do nothing
    for idx, list in enumerate(factor_lists):
        if idx%2 == 1:
            factor_lists[idx] = factor_lists[idx][::-1] 

    factor_lists = np.array(factor_lists).T
    prod_block_sizes = np.ones((len(factor_lists)+1, len(factor_lists[0])), dtype = int)
    prod_block_sizes[1:-1] = np.cumprod(factor_lists[-1:0:-1], axis =0)[::-1]
    prod_block_sizes[0] = prod_block_sizes[0] * 1e100

    return factor_lists, prod_block_sizes

@time_function
def hierarchical_block_indexing(index, prod_block_sizes):
    return np.floor(np.mod(index.reshape([1]+list(index.shape)), prod_block_sizes[:-1].reshape(list(prod_block_sizes[:-1].shape)+[1]*(prod_block_sizes.shape[1])))/prod_block_sizes[1:].reshape(list(prod_block_sizes[1:].shape)+[1]*(prod_block_sizes.shape[1]))).astype(int)

def gen_encoding_map(shape):
    dim = len(shape)
    block_sizes, prod_blocks = get_factorlist(shape)
    indices_all = np.indices(shape)
    mapped_indexes = hierarchical_block_indexing(indices_all, prod_blocks)
    final_map = np.empty([len(block_sizes)]+list(shape))
    for i in range(len(block_sizes)):
        final_map[i] = np.ravel_multi_index(mapped_indexes[i], block_sizes[i])
    return np.prod(block_sizes, axis= 1), final_map.astype(int)

In [14]:
indices_all = np.indices(shape)

In [15]:
mapped_indexes = hierarchical_block_indexing(indices_all, prod_blocks)

Time to run hierarchical_block_indexing: 1.16085100 seconds


In [16]:
mapped_indexes.shape

(4, 3, 240, 240, 220)

In [17]:
block_sizes, prod_blocks = get_factorlist((240,240,220))

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


In [18]:
np.prod(block_sizes, axis =1)

array([ 30,  32,  80, 165])

In [21]:
qubit_size, kron = get_block_encoding_map_general((240,240,220))

In [22]:
fin_map = gen_encoding_map((240,240,220))

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


Time to run hierarchical_block_indexing: 1.23170495 seconds


In [23]:
fin_map.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [24]:
fin_map[:, 100,100,100]

TypeError: tuple indices must be integers or slices, not tuple

In [25]:
fin_map_moved = np.moveaxis(fin_map, 0, -1)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 4) + inhomogeneous part.

In [29]:
fin_map_moved.shape

(240, 240, 220, 4)

In [33]:
fin_map_moved[100,100,100]

array([14.,  9.,  9., 12.])

In [65]:
class NDMPS_new:
    def __init__(self, mps=None, qubit_size=None, encoding_map=None, norm=True, mode="Std", min_value = 0, max_value = 1):
        self.qubit_size = qubit_size
        self.encoding_map = encoding_map
        self.mps = mps
        self.norm = norm #Normalize matrix data
        #Compression mode 
        # "Std" standard Block Encoding
        # "DCT" discrete cosine fourier transform before compression
        self.mode = mode 
        self.min_value = min_value
        self.max_value = max_value
    
    @classmethod
    # @time_function
    def from_matrix(cls, matrix, norm = False, mode = "Std"):
        qubit_size, encoding_map = gen_encoding_map(matrix.shape)
        encoding_map = np.moveaxis(encoding_map, 0, -1)

        #check for flags
        if norm:
            matrix = matrix / (np.linalg.norm(matrix))
        if mode == "DCT":
            matrix = dct(matrix, norm = "ortho")

        #initialize tensor
        contracted_tensor = np.empty(shape = tuple(qubit_size))


        #encode matrix data
        # start_nested_loop = time.time()
        it = np.nditer(matrix, flags=['multi_index'])
        for _ in it:
            contracted_tensor[tuple(encoding_map[it.multi_index])] = matrix[it.multi_index]
        
        # nested_loop_time = time.time() - start_nested_loop
        # print(f"Time for nested loops: {nested_loop_time:.4f} seconds")
        #put in MPS
        # start_mps_creation = time.time()
        mps = qtn.MatrixProductState.from_dense(contracted_tensor, dims = tuple(qubit_size))
        # mps_creation_time = time.time() - start_mps_creation
        # print(f"Time to create MPS from dense tensor: {mps_creation_time:.4f} seconds")
        #return class
        return cls(mps, qubit_size, encoding_map, norm, mode, np.min(matrix), np.max(matrix))

    # @time_function
    def compression_ratio(self):
        initial_N = np.prod(self.qubit_size)
        compressed_N = self.number_elements_in_MPS()
        # TODO: also implement the compression rate in bits / bits
        return compressed_N / initial_N
        
    # @time_function
    def compress(self, cutoff):
        """
        Compresses a Matrix Product State (MPS) by cutting bonds with a relative cutoff value.
        Arguments:
            cutoff (float): The relative cutoff value to use for bond compression.
        Returns:
            None
        """
        size = len(self.mps.sites)
        for i in np.arange(1, size):
            t1 = self.mps[i-1] # Tensor 1
            t2 = self.mps[i] # Tensor 2
            # Compress bond according to percentage * bond dimension
            qtn.tensor_compress_bond(t1, t2, cutoff = cutoff, cutoff_mode = "rel") 
    def continuous_compress(self, cutoff, print_ratio = True):
        compress_list = np.array([0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1]) * cutoff
        for c in compress_list:
            self.compress(c)
            if print_ratio:
                print(f"Compression ratio at {c}: {self.compression_ratio()}")


    # @time_function
    def number_elements_in_MPS(self):
        """
        Returns the number of tensor elements in the quimb MPS.
        Parameters:
            mps: quimb MatrixProductState object
        Returns:
            int: The total number of tensor elements in the MPS."""
        return sum(t.size for t in self.mps)
    
    # @time_function
    def mps_to_matrix(self):
        """
        Converts the compressed Matrix Product State (MPS) representation back to an image matrix.
        Arguments:
            None
        Returns:
            Compressed matrix
        """

        #conract mps
        contracted_mps = self.mps ^ ...

        #order tensor legs back
        for i in np.arange(len(contracted_mps.inds)):
            contracted_mps.moveindex("k"+str(i), i, inplace=True)
        
        #return in correct format

        recovered_tensor = np.empty(self.encoding_map.shape)
        it = np.nditer(recovered_tensor, flags=['multi_index'])
        for _ in it:
            recovered_tensor[it.multi_index] = contracted_mps.data[self.encoding_map[it.multi_index]]
        
        if self.mode == "Std":
            return recovered_tensor
        elif self.mode == "DCT":
            return idct(recovered_tensor, norm = "ortho")

In [66]:
test_tens = np.random.rand(240,240,220)

In [67]:
test_tens.shape

(240, 240, 220)

In [68]:
mps_test = NDMPS_new.from_matrix(test_tens)

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


Time to run hierarchical_block_indexing: 1.29798508 seconds


In [70]:
mps_old = NDMPS.from_matrix(test_tens)

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [30]:
test_tens = np.random.rand(240,240,220)

In [31]:
qubit_size, encoding_map = gen_encoding_map(test_tens.shape)
encoding_map = np.moveaxis(encoding_map, 0, -1)

contracted_tensor = np.empty(shape = tuple(qubit_size))
print(contracted_tensor.shape)


/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


Time to run hierarchical_block_indexing: 1.19645214 seconds
(30, 32, 80, 165)


In [32]:
it = np.nditer(test_tens, flags=['multi_index'])
for _ in it:
    contracted_tensor[tuple(encoding_map[it.multi_index])] = test_tens[it.multi_index]

In [33]:
# Convert multi-index encoding_map to flat indices
flat_indices = np.ravel_multi_index(encoding_map.T, test_tens.shape)

# Flatten the input matrix
flattened_matrix = test_tens.ravel()

# Reorder data efficiently
contracted_tensor = flattened_matrix[flat_indices].reshape(contracted_tensor.shape)

ValueError: parameter multi_index must be a sequence of length 3

In [34]:
test_tens.shape

(240, 240, 220)

In [35]:
contracted_tensor[tuple(encoding_map)]

IndexError: too many indices for array

In [36]:
encoding_map.shape

(240, 240, 220, 4)

In [37]:
contracted_tensor[np.array([0,0,0,30])]

IndexError: index 30 is out of bounds for axis 0 with size 30

In [38]:
mps = qtn.MatrixProductState.from_dense(contracted_tensor, dims = tuple(qubit_size))

In [39]:
block_sizes[0]

array([3, 5, 2])

In [40]:
mapped_indexes[0].reshape(3,220*220*240)

ValueError: cannot reshape array of size 38016000 into shape (3,11616000)

In [41]:
mapped_indexes[0].min()

np.int64(0)

In [42]:
block_sizes

array([[ 3,  5,  2],
       [ 4,  4,  2],
       [ 4,  4,  5],
       [ 5,  3, 11]])

In [43]:
mapped_indexes[0,:,100,100,100]

array([1, 2, 0])

In [44]:
tuple(block_sizes[0])

(np.int64(3), np.int64(5), np.int64(2))

In [45]:
np.ravel_multi_index(mapped_indexes[3,:,100,100,100].astype(int), block_sizes[3])

np.int64(12)

In [46]:
np.array(index).shape

(3,)

In [47]:
(1,2)+(1)

TypeError: can only concatenate tuple (not "int") to tuple

In [48]:
[np.inf]*5

[inf, inf, inf, inf, inf]

In [49]:
print(block_sizes)
print("-----")
print(prod_blocks)

[[ 3  5  2]
 [ 4  4  2]
 [ 4  4  5]
 [ 5  3 11]]
-----
[[9223372036854775807 9223372036854775807 9223372036854775807]
 [                 80                  48                 110]
 [                 20                  12                  55]
 [                  5                   3                  11]
 [                  1                   1                   1]]


In [50]:
def hierarchical_block_indexing(index, prod_block_sizes):
    return np.floor(np.mod(index, prod_block_sizes[:-1])/prod_block_sizes[1:])

In [51]:
np.mod(index, prod_blocks[:-1])

array([[100, 100, 100],
       [ 20,   4, 100],
       [  0,   4,  45],
       [  0,   1,   1]])

In [52]:
np.mod(100,np.inf)

np.float64(100.0)

In [53]:
index/np.prod(block_sizes[1:], axis=0)

array([1.25      , 2.08333333, 0.90909091])

In [54]:
def hierarchical_block_indexing_old(index, shape, block_sizes):
    num_levels = len(block_sizes)
    dim = len(index)
    # important only insert numpy arrays
    hierarchical_indices = np.zeros((num_levels, dim))
    for level in range(num_levels):
        if level == 0:
            hierarchical_indices[0] = (np.floor(index/np.prod(block_sizes[1:], axis=0)))
        elif level == num_levels-1:
            hierarchical_indices[level] = (np.floor(np.mod(index, block_sizes[-1])))
        else:
            hierarchical_indices[level]= (np.floor(np.mod(index, np.prod(block_sizes[level:], axis=0))/np.prod(block_sizes[level+1:], axis=0)))
    return hierarchical_indices

In [55]:
shape = (220,220,220)
index = (100, 100,100)
block_sizes = get_factorlist(shape)
stdId_to_hierarchical_index(index, shape, block_sizes)

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


TypeError: hierarchical_block_indexing() takes 2 positional arguments but 3 were given

In [56]:
(6e-5 * 220**3)/60

10.648

In [57]:
size = 40
A = np.random.rand(size,size,size)
mps_old = NDMPS_old.from_matrix(A, mode = "DCT")

In [58]:
mps_new = NDMPS.from_matrix(A, mode = "DCT")

/var/folders/2k/n_pr2lyj52ggrhmgv2zx_jt40000gn/T/ipykernel_6316/2243089732.py:39: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [59]:
mps_old.mps @ mps_new.mps

NameError: name 'mps_new' is not defined

In [60]:
mps_old.mps @ mps_old.mps

21251.431561147743

In [129]:
get_block_encoding_map_general((220,220,240))

([66, 40, 40, 110],
 array([[[(np.int64(0), np.int64(0), np.int64(0), np.int64(0)),
          (np.int64(0), np.int64(0), np.int64(0), np.int64(1)),
          (np.int64(0), np.int64(0), np.int64(0), np.int64(2)), ...,
          (np.int64(2), np.int64(3), np.int64(3), np.int64(2)),
          (np.int64(2), np.int64(3), np.int64(3), np.int64(3)),
          (np.int64(2), np.int64(3), np.int64(3), np.int64(4))],
         [(np.int64(0), np.int64(0), np.int64(0), np.int64(5)),
          (np.int64(0), np.int64(0), np.int64(0), np.int64(6)),
          (np.int64(0), np.int64(0), np.int64(0), np.int64(7)), ...,
          (np.int64(2), np.int64(3), np.int64(3), np.int64(7)),
          (np.int64(2), np.int64(3), np.int64(3), np.int64(8)),
          (np.int64(2), np.int64(3), np.int64(3), np.int64(9))],
         [(np.int64(0), np.int64(0), np.int64(4), np.int64(0)),
          (np.int64(0), np.int64(0), np.int64(4), np.int64(1)),
          (np.int64(0), np.int64(0), np.int64(4), np.int64(2)), ...,
   

In [2]:
factorlist , prod_blocks = get_factorlist((220,220,240))

/Users/maxge/Documents/Studium/München/02_SS 2024/QEL/Block encoding generalization/img-compression-mps/utils_ND.py:84: RuntimeWarning: invalid value encountered in cast
  prod_block_sizes[0] = prod_block_sizes[0] * 1e100


In [3]:
factorlist

array([[ 2, 11,  3],
       [ 2,  5,  4],
       [ 5,  2,  4],
       [11,  2,  5]])

In [4]:
prod_blocks

array([[9223372036854775807, 9223372036854775807, 9223372036854775807],
       [                110,                  20,                  80],
       [                 55,                   4,                  20],
       [                 11,                   2,                   5],
       [                  1,                   1,                   1]])

In [5]:
2*2*5*11

220